In [9]:
# import dependencies
import py_midicsv as pm


In [10]:
# define functions

# rescale between two ranges of values
def scale(OldValue, OldMin, OldMax, NewMin, NewMax):
    NewValue = (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
    return NewValue

# convert time in ms to time in however the midi file wants it
def ms2midi(ms):
    new_ms = scale(ms, 0, 500, 0, 480)
    return new_ms

# convert seconds and frames to ms
def frame2ms(t):
    return int(t)*1000 + round(round((t - int(t))*100)*(1000/framerate))


In [11]:
# names for imported and exported files
og_midi = "bells.mid"
new_midi = "remappedharp.mid"
convert_csv = "example_converted.csv"

# set framerate
framerate = 28.67

# set bpm
bpm = 120


In [12]:
# Load the MIDI file and parse it into CSV format
csv_string = pm.midi_to_csv(og_midi)

# display csv_string
csv_string


['0, 0, Header, 0, 1, 480\n',
 '1, 0, Start_track\n',
 '1, 84960, Channel_prefix, 0\n',
 '1, 84960, Title_t, "Broken Bells"\n',
 '1, 84960, Instrument_name_t, "Broken Bells"\n',
 '1, 84960, Time_signature, 4, 2, 24, 8\n',
 '1, 84960, Key_signature, 0, "major"\n',
 '1, 84960, SMPTE_offset, 33, 0, 0, 0, 0\n',
 '1, 84960, Tempo, 500000\n',
 '1, 84960, Note_on_c, 0, 62, 98\n',
 '1, 84960, Note_on_c, 0, 66, 98\n',
 '1, 84960, Note_on_c, 0, 71, 98\n',
 '1, 86400, Note_on_c, 0, 72, 98\n',
 '1, 87484, Note_on_c, 0, 72, 0\n',
 '1, 87510, Note_off_c, 0, 66, 64\n',
 '1, 87567, Note_on_c, 0, 71, 0\n',
 '1, 87580, Note_on_c, 0, 62, 0\n',
 '1, 87840, Note_on_c, 0, 71, 98\n',
 '1, 90447, Note_off_c, 0, 71, 64\n',
 '1, 90720, Note_on_c, 0, 81, 98\n',
 '1, 92640, Note_off_c, 0, 81, 64\n',
 '1, 93600, Note_on_c, 0, 59, 98\n',
 '1, 93600, Note_on_c, 0, 78, 98\n',
 '1, 95040, Note_off_c, 0, 78, 64\n',
 '1, 96150, Note_off_c, 0, 59, 64\n',
 '1, 96480, Note_on_c, 0, 62, 98\n',
 '1, 96480, Note_on_c, 0, 66, 

In [13]:
# retrive note times from midi csv string

# list to hold note times
times = []

# extract not times as integers from string
for line in csv_string:
    if 'Note_' in line or 'End_track' in line:
        time = int(line.split(',')[1][1:])
        times.append(time)

# note times relative to first note in midi track (not relative to begining of song file)
times = list(map(lambda t: t - times[0], times))



In [14]:
# get midi timings for steps

# list of step timings
timings = [
    0.01, 0.25, 1.20, 2.10, 3.00, 3.28, 4.21, 5.11, 6.08, 7.02, 8.04, 8.26, 9.19, 10.13,
    11.05, 12.03, 12.27, 13.16, 14.11, 15.05, 15.27, 16.21, 17.17, 18.10, 19.03, 20.00, 
    20.23, 21.26, 23.12, 24.14, 25.10, 26.02, 27.01, 27.25, 28.19, 29.12, 30.08, 31.02, 
    31.19, 32.11, 33.06, 34.01, 34.21, 35.15, 36.10, 37.04, 37.25, 38.20, 39.14, 40.10, 
    41.01, 41.27, 42.21, 43.14, 44.08, 45.01, 45.24, 46.22, 47.12, 48.04
]

# convert seconds and frames to ms
timings = list(map(frame2ms, timings))

# # convert ms to midi times
# timings = list(map(ms2midi, timings))


In [21]:
timings.insert(0, 0)

In [22]:
timings

[0,
 35,
 872,
 1698,
 2349,
 3000,
 3977,
 4732,
 5384,
 6279,
 7070,
 8140,
 8907,
 9663,
 10453,
 11174,
 12105,
 12942,
 13558,
 14384,
 15174,
 15942,
 16732,
 17593,
 18349,
 19105,
 20000,
 20802,
 21907,
 23419,
 24488,
 25349,
 26070,
 27035,
 27872,
 28663,
 29419,
 30279,
 31070,
 31663,
 32384,
 33209,
 34035,
 34732,
 35523,
 36349,
 37140,
 37872,
 38698,
 39488,
 40349,
 41035,
 41942,
 42732,
 43488,
 44279,
 45035,
 45837,
 46767,
 47419,
 48140]

In [23]:


for i in range(1, len(timings)):
    
    w = i*500
    
    
    print(f'{i}, {w} {timings[i] - timings[i-1]};')




1, 500 35;
2, 1000 837;
3, 1500 826;
4, 2000 651;
5, 2500 651;
6, 3000 977;
7, 3500 755;
8, 4000 652;
9, 4500 895;
10, 5000 791;
11, 5500 1070;
12, 6000 767;
13, 6500 756;
14, 7000 790;
15, 7500 721;
16, 8000 931;
17, 8500 837;
18, 9000 616;
19, 9500 826;
20, 10000 790;
21, 10500 768;
22, 11000 790;
23, 11500 861;
24, 12000 756;
25, 12500 756;
26, 13000 895;
27, 13500 802;
28, 14000 1105;
29, 14500 1512;
30, 15000 1069;
31, 15500 861;
32, 16000 721;
33, 16500 965;
34, 17000 837;
35, 17500 791;
36, 18000 756;
37, 18500 860;
38, 19000 791;
39, 19500 593;
40, 20000 721;
41, 20500 825;
42, 21000 826;
43, 21500 697;
44, 22000 791;
45, 22500 826;
46, 23000 791;
47, 23500 732;
48, 24000 826;
49, 24500 790;
50, 25000 861;
51, 25500 686;
52, 26000 907;
53, 26500 790;
54, 27000 756;
55, 27500 791;
56, 28000 756;
57, 28500 802;
58, 29000 930;
59, 29500 652;
60, 30000 721;


In [8]:
# remap notes in midi file to synchronise with steps as though steps were quarter notes

# which note in csv_string
which = 0

# rescaled times for each note
new_times = []

# get duration of quarter note in ms according to set bpm
bpm_ms = ms2midi(60000/bpm)

# loop through note times
for time in times:
    
    # calculate how many quarter notes from the beginning the note is
    beat = int(time/bpm_ms)
    
    # origninal minimum and maximum for scaling
    omin = beat*bpm_ms
    omax = (beat+1)*bpm_ms
    
    # new minimum and maximum for scaling
    try:
        nmin = timings[beat]
        nmax = timings[beat+1]
    
    # if note is on a beat that exceeds the number of steps
    except IndexError:
        
        # new min is second to last step new max is last step
        nmin = timings[len(timings)-2]
        nmax = timings[len(timings)-1]
    
    # rescaled time in string format
    new_time = ' ' + str(int(scale(time, omin, omax, nmin, nmax)))
    
    # add rescaled time
    new_times.append(new_time)


In [123]:
# time = times[40]
# time

In [124]:
# time = times[41]

# # calculate how many quarter notes from the beginning the note is
# beat = int(time/bpm_ms)

# # origninal minimum and maximum for scaling
# omin = beat*bpm_ms
# omax = (beat+1)*bpm_ms

# # new minimum and maximum for scaling
# nmin = timings[beat]
# nmax = timings[beat+1]

# # rescaled time in string format
# new_time = ' ' + str(int(scale(time, omin, omax, nmin, nmax)))

# # add rescaled time
# print(beat)
# print(new_time)

In [125]:
for i, time in enumerate(new_times):
    print(f'{i}: {time}')

0:  33
1:  33
2:  33
3:  33
4:  4542
5:  4542
6:  4542
7:  4542
8:  4542
9:  4558
10:  4627
11:  4674
12:  9095
13:  9095
14:  9095
15:  9095
16:  9276
17:  9276
18:  9276
19:  9276
20:  13697
21:  13727
22:  13808
23:  13808
24:  13808
25:  13808
26:  13841
27:  13854
28:  18029
29:  18076
30:  18085
31:  18112
32:  18340
33:  18340
34:  18340
35:  18340
36:  24335
37:  24335
38:  24335
39:  24335
40:  24335
41:  24352
42:  24428
43:  24480
44:  28861
45:  28861
46:  28861
47:  28861
48:  29067
49:  29067
50:  29067
51:  29067
52:  33249
53:  33274
54:  33342
55:  33342
56:  33342
57:  33342
58:  33375
59:  33388
60:  37583
61:  37631
62:  37641
63:  37669
64:  37908
65:  37908
66:  37908
67:  37908
68:  42507
69:  42507
70:  42507
71:  42507
72:  42507
73:  42525
74:  42606
75:  42660
76:  46041
77:  46041
78:  46041
79:  46041
80:  45522
81:  45522
82:  45522
83:  45522
84:  46117
85:  46143
86:  45522
87:  45522
88:  45522
89:  45522
90:  45552
91:  45564
92:  45917
93:  45962
94: 

In [126]:
for i, time in enumerate(timings):
    print(f'{i}: {time}')

0: 33.6
1: 837.12
2: 1630.08
3: 2255.04
4: 2880.0
5: 3817.92
6: 4542.72
7: 5168.64
8: 6027.84
9: 6787.2
10: 7814.4
11: 8550.72
12: 9276.48
13: 10034.88
14: 10727.04
15: 11620.8
16: 12424.32
17: 13015.68
18: 13808.64
19: 14567.04
20: 15304.32
21: 16062.72
22: 16889.28
23: 17615.04
24: 18340.8
25: 19200.0
26: 19969.92
27: 21030.72
28: 22482.24
29: 23508.48
30: 24335.04
31: 25027.2
32: 25953.6
33: 26757.12
34: 27516.48
35: 28242.24
36: 29067.84
37: 29827.2
38: 30396.48
39: 31088.64
40: 31880.64
41: 32673.6
42: 33342.72
43: 34102.08
44: 34895.04
45: 35654.4
46: 36357.12
47: 37150.08
48: 37908.48
49: 38735.04
50: 39393.6
51: 40264.32
52: 41022.72
53: 41748.48
54: 42507.84
55: 43233.6
56: 44003.52
57: 44896.32
58: 45522.24
59: 46214.4


In [145]:
which = 0

for i, line in enumerate(csv_string):
    if 'Note_' in line or 'End_track' in line:
        new_line = line.split(',')
        new_line[1] = new_times[which]
        csv_string[i] = ','.join(map(str,new_line))
        which += 1
        
csv_string
        

['0, 0, Header, 0, 1, 480\n',
 '1, 0, Start_track\n',
 '1, 84960, Channel_prefix, 0\n',
 '1, 84960, Title_t, "Broken Bells"\n',
 '1, 84960, Instrument_name_t, "Broken Bells"\n',
 '1, 84960, Time_signature, 4, 2, 24, 8\n',
 '1, 84960, Key_signature, 0, "major"\n',
 '1, 84960, SMPTE_offset, 33, 0, 0, 0, 0\n',
 '1, 84960, Tempo, 500000\n',
 '1, 33, Note_on_c, 0, 62, 98\n',
 '1, 33, Note_on_c, 0, 66, 98\n',
 '1, 33, Note_on_c, 0, 71, 98\n',
 '1, 2255, Note_on_c, 0, 72, 98\n',
 '1, 4005, Note_on_c, 0, 72, 0\n',
 '1, 4044, Note_off_c, 0, 66, 64\n',
 '1, 4130, Note_on_c, 0, 71, 0\n',
 '1, 4150, Note_on_c, 0, 62, 0\n',
 '1, 4542, Note_on_c, 0, 71, 98\n',
 '1, 8863, Note_off_c, 0, 71, 64\n',
 '1, 9276, Note_on_c, 0, 81, 98\n',
 '1, 12424, Note_off_c, 0, 81, 64\n',
 '1, 13808, Note_on_c, 0, 59, 98\n',
 '1, 13808, Note_on_c, 0, 78, 98\n',
 '1, 16062, Note_off_c, 0, 78, 64\n',
 '1, 17841, Note_off_c, 0, 59, 64\n',
 '1, 18340, Note_on_c, 0, 62, 98\n',
 '1, 18340, Note_on_c, 0, 66, 98\n',
 '1, 18340

In [146]:
with open(convert_csv, "w") as f:
    f.writelines(csv_string)

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open(new_midi, "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)


In [ ]:
len(times)

In [ ]:


# timings = [0]

# for i in range(len(times)):
#     if i % 3 == 0:
#         timings.append(timings[i]+miditime(500))
#     else:
#         timings.append(timings[i]+miditime(250))


In [ ]:
# timings = [810, 1561, 2362, 3041, 3346, 3565, 4394, 4919, 5336, 6195, 7149, 7381, 8240, 8575, 9540, 9803]
# timings = [190, 620, 960, 1400, 1780, 2160, 2340, 3340, 3420, 4250, 5120, 6000]
# timings =  [80, 270, 420, 1170, 1330, 2050, 2150, 3140, 3190, 4100, 5050, 6000]
# timings =  [.08, .27, .42, 1.17, 1.33, 2.05, 2.15, 3.14, 3.19, 4.10, 5.05, 6.00]

In [ ]:
# test_times = [0]


# for i in range(len(times)):
#     if i % 3 == 0:
#         test_times.append(test_times[i]+480)
#     else:
#         test_times.append(test_times[i]+480)

# test_times.pop(len(test_times)-1)

# test_times

    

In [ ]:
# # timings = [0]

# # for i in range(len(test_times)):
# #     if i % 2 == 0:
# #         timings.append(timings[i]+1000)
# #     else:
# #         timings.append(timings[i]+2000)

# timings = [0]

# # for i in range(len(test_times)):
# #     if i % 3 == 0:
# #         timings.append(timings[i]+480)
# #     else:
# #         timings.append(timings[i]+480)

# for i in range(len(times)):
#     if i % 3 == 0:
#         timings.append(timings[i]+480)
#     else:
#         timings.append(timings[i]+(480/2))
        

In [ ]:
# which = 0

# new_times = []

# bpm_ms = 480

# # for i, time in enumerate(test_times):
    
# #     beat = int(time/bpm_ms)
    
# #     omin = beat*bpm_ms
# #     omax = (beat+1)*bpm_ms
    
# #     try:
# #         nmin = timings[beat]
# #         nmax = timings[beat+1]
        
# #     except IndexError:
        
# #         nmin = timings[len(timings)-1]
# #         nmax = timings[len(timings)-1]
    
# #     new_time = ' ' + str(int(scale(time, omin, omax, nmin, nmax)))
    
# #     new_times.append(new_time)

# # new_times

# for i, time in enumerate(times):
    
#     beat = int(time/bpm_ms)
    
#     omin = beat*bpm_ms
#     omax = (beat+1)*bpm_ms
    
#     try:
#         nmin = timings[beat]
#         nmax = timings[beat+1]
        
#     except IndexError:
        
#         nmin = timings[len(timings)-1]
#         nmax = timings[len(timings)-1]
    
#     new_time = ' ' + str(int(scale(time, omin, omax, nmin, nmax)))
    
#     new_times.append(new_time)

# new_times


In [ ]:
# Load the MIDI file and parse it into CSV format
csv_string = pm.midi_to_csv("steps.mid")

csv_string


In [ ]:




for i, line in enumerate(csv_string):
    if 'Note_' in line:
        new_line = line.split(',')
        new_line[1] = ' ' + str(int(new_line[1][1:])*2)
        csv_string[i] = ','.join(map(str,new_line))
        which += 1
        
csv_string


In [ ]:
with open("example_converted.csv", "w") as f:
    f.writelines(csv_string)

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("example_converted.mid", "wb") as output_file:
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)


In [ ]:
new_times = []

bpm_ms = 500

for i, time in enumerate(times):
    
    beat = int(timing/bpm_ms)

    time = scale(timing, omin, omax, nmin, nmax)
    print(time)



In [ ]:
timings = [0]

for i in range(len(times)):
    if i % 3 == 0:
        timings.append(timings[i]+500)
    else:
        timings.append(timings[i]+250)

In [ ]:
with open("example_converted.csv", "w") as f:
    f.writelines(csv_string)
    
    

In [ ]:
# for time in events[" 0"]:
#     print(time)

# bpm_ms = 500

# new_events = events.copy()

# for index in new_events.index:
#     timing = new_events.loc[index][" 0"]
#     beat = int(timing/500)
#     print(timings[beat]/500)
#     time = int(timings[beat]/500 * timing)
#     new_events[" 0"][index] = time
    

In [ ]:
events = midi.loc[8:139]
events

In [ ]:
scale(50, 0, 100, 0, 1000)

In [ ]:
bpm_ms = 500

events2 = events.copy()

new_times = []

for i, timing in enumerate(new_events[" 0"]):

    beat = int(timing/bpm_ms)
    
    omin = beat*bpm_ms
    omax = (beat+1)*bpm_ms
    nmin = timings[beat]
    nmax = timings[beat+1]
    
    time = scale(timing, omin, omax, nmin, nmax)
    print(time)
    
    
    
    
#     within = timing/500 - beat*bpm_ms


In [ ]:
new_events.head(20)

In [ ]:
events.head(20)

In [ ]:
midi.loc[8:27]

In [ ]:
events.head(20)